In [107]:
import numpy as np
import pandas as pd
import math
import os
import sklearn.preprocessing

In [7]:
mail = pd.read_csv('mails.csv')

In [11]:
length = len(mail)

In [17]:
source_set = set(mail['source'].unique())
target_set = set(mail['target'].unique())

In [20]:
people = source_set.union(target_set)

In [23]:
G = np.zeros((len(people),len(people)))

In [123]:
for i,m in mail.iterrows():
    G[m.source,m.target] = m.weight
    G[m.target,m.source] = m.weight

In [124]:
np.fill_diagonal(G,1)

In [119]:
def inflation(G,r):
    return sklearn.preprocessing.normalize(np.power(G,r),norm='l1',axis=0)

In [109]:
def expansion(G,e):
    return G**e

In [110]:
def convergence(G,T):
    return np.sum(np.abs(G-T)) < 0.00001

In [120]:
def MCL(G,e,r):
    while True:
        I = expansion(G,e)
        I = inflation(I,r)
        if convergence(G,I):
            break
        G = I
    return I

In [131]:
np.set_printoptions(precision=1,suppress=True)

In [128]:
a = MCL(G,2,2)

In [143]:
for row in range(a.shape[0]):
    for column in range(a.shape[1]):
        G_out.append([row, column, a[row, column]])

[[0, 0, 0.3333333333333333],
 [0, 1, 0.3333333333333333],
 [0, 2, 0.0],
 [0, 3, 0.0],
 [0, 4, 0.0],
 [0, 5, 0.0],
 [0, 6, 0.0],
 [0, 7, 0.0],
 [0, 8, 0.0],
 [0, 9, 0.0],
 [0, 10, 0.0],
 [0, 11, 0.0],
 [0, 12, 0.0],
 [0, 13, 0.0],
 [0, 14, 0.0],
 [0, 15, 0.0],
 [0, 16, 0.0],
 [0, 17, 0.0],
 [1, 0, 0.3333333333333333],
 [1, 1, 0.3333333333333333],
 [1, 2, 0.0],
 [1, 3, 0.0],
 [1, 4, 0.0],
 [1, 5, 0.0],
 [1, 6, 0.0],
 [1, 7, 0.0],
 [1, 8, 0.0],
 [1, 9, 0.0],
 [1, 10, 0.0],
 [1, 11, 0.0],
 [1, 12, 0.0],
 [1, 13, 0.0],
 [1, 14, 0.0],
 [1, 15, 0.0],
 [1, 16, 0.0],
 [1, 17, 0.0],
 [2, 0, 0.3333333333333333],
 [2, 1, 0.3333333333333333],
 [2, 2, 0.0],
 [2, 3, 0.0],
 [2, 4, 0.0],
 [2, 5, 3.3562198154764474e-228],
 [2, 6, 0.0],
 [2, 7, 0.0],
 [2, 8, 0.0],
 [2, 9, 0.0],
 [2, 10, 0.0],
 [2, 11, 0.0],
 [2, 12, 0.0],
 [2, 13, 0.0],
 [2, 14, 0.0],
 [2, 15, 0.0],
 [2, 16, 0.0],
 [2, 17, 0.0],
 [3, 0, 0.0],
 [3, 1, 0.0],
 [3, 2, 4.8144948555348436e-181],
 [3, 3, 0.0],
 [3, 4, 0.0],
 [3, 5, 0.0],
 [3, 6

In [145]:
columns_name = ['source','target','weight']
G_out = pd.DataFrame(G_out,columns=columns_name)

In [147]:
G_out.to_json(orient='records')

'[{"source":0,"target":0,"weight":0.3333333333},{"source":0,"target":1,"weight":0.3333333333},{"source":0,"target":2,"weight":0.0},{"source":0,"target":3,"weight":0.0},{"source":0,"target":4,"weight":0.0},{"source":0,"target":5,"weight":0.0},{"source":0,"target":6,"weight":0.0},{"source":0,"target":7,"weight":0.0},{"source":0,"target":8,"weight":0.0},{"source":0,"target":9,"weight":0.0},{"source":0,"target":10,"weight":0.0},{"source":0,"target":11,"weight":0.0},{"source":0,"target":12,"weight":0.0},{"source":0,"target":13,"weight":0.0},{"source":0,"target":14,"weight":0.0},{"source":0,"target":15,"weight":0.0},{"source":0,"target":16,"weight":0.0},{"source":0,"target":17,"weight":0.0},{"source":1,"target":0,"weight":0.3333333333},{"source":1,"target":1,"weight":0.3333333333},{"source":1,"target":2,"weight":0.0},{"source":1,"target":3,"weight":0.0},{"source":1,"target":4,"weight":0.0},{"source":1,"target":5,"weight":0.0},{"source":1,"target":6,"weight":0.0},{"source":1,"target":7,"weigh